In [1]:
!pip install sentence_transformers
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.3 MB/s eta 0:00:0000:0100:01


# 1) Data Loading & Pre-processing

In [2]:
import pandas as pd
df = pd.read_excel("/kaggle/input/nlp-project-dataset/Main.xlsx")

In [3]:
df.head()

,Batch,Source Language,ID,Type,Text,English MT,Arabic MT,Annotator ID,Bias,Propaganda,Type of Propaganda,Type of Bias,Comments
0,B01,English,1,MAIN,Yemen's Houthis have waded into the Israel-Ham...,Yemen's Houthis have waded into the Israel-Ham...,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,1.0,Biased against Palestine,Not Propaganda,Propaganda Not to be deleted,ضمني,NaN
1,B01,English,2,MAIN,Isreal - Hamas Conflict | Face to Face,Isreal - Hamas Conflict | Face to Face,إسرائيل - الصراع مع حماس | وجها لوجه,4.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
2,B01,English,4,MAIN,Videos show how armed men from Gaza stormed a ...,Videos show how armed men from Gaza stormed a ...,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,3.0,Unbiased,Not Propaganda,Propaganda Not to be deleted,NaN,NaN
3,B01,English,7,MAIN,Protest in Aligarh Muslim University in suppor...,Protest in Aligarh Muslim University in suppor...,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,5.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
4,B01,English,8,MAIN,IDF releases audio recording about misfired ro...,IDF releases audio recording about misfired ro...,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,2.0,Biased against Palestine,Propaganda,Propaganda Must be deleted,ضمني,NaN


In [4]:
df.columns

Index(['Batch', 'Source Language', 'ID', 'Type', 'Text', 'English MT',
       'Arabic MT', 'Annotator ID', 'Bias', 'Propaganda', 'Type of Propaganda',
       'Type of Bias', 'Comments'],
      dtype='object')

In [5]:
df = df.drop(columns=['Annotator ID', 'Text','Batch', 'ID', 'Comments', 'Type', 'Source Language', 'English MT',"Type of Bias","Propaganda","Type of Propaganda"], errors='ignore')

In [6]:
df.head()

,Arabic MT,Bias
0,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,Biased against Palestine
1,إسرائيل - الصراع مع حماس | وجها لوجه,Unbiased
2,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,Unbiased
3,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,Unbiased
4,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,Biased against Palestine


In [7]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 13500


In [8]:
df.isna().sum()

Arabic MT       0
Bias         2700
dtype: int64

In [9]:
import numpy as np

In [10]:
df.replace(['', 'None', 'nan'], np.nan, inplace=True)  
df.dropna(subset=['Bias'], inplace=True)

In [11]:
df.isna().sum()

Arabic MT    0
Bias         0
dtype: int64

In [12]:
df.head()

,Arabic MT,Bias
0,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,Biased against Palestine
1,إسرائيل - الصراع مع حماس | وجها لوجه,Unbiased
2,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,Unbiased
3,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,Unbiased
4,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,Biased against Palestine


In [13]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 10800


In [14]:
print(df['Bias'].cat.categories if df['Bias'].dtype.name == 'category' else df['Bias'].unique())

['Biased against Palestine' 'Unbiased' 'Unclear' 'Biased against others'
 'Biased against Israel' 'Biased against both Palestine and Israel'
 'Not Applicable']


In [15]:
columns_to_check = ['Bias']

for column in columns_to_check:
    print(f"Class counts for '{column}':")
    print(df[column].value_counts())
    print("\n")

Class counts for 'Bias':
Bias
Unbiased                                    6817
Biased against Palestine                    2900
Unclear                                      432
Biased against Israel                        281
Biased against others                        203
Not Applicable                               120
Biased against both Palestine and Israel      47
Name: count, dtype: int64




In [16]:
!pip install emoji

In [17]:
import re
import emoji

def preprocess_text(text):
    # Ensure text is a string
    text = str(text)
    
    # 1. Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # 2. Remove full URLs with protocols (e.g., "http://example.com")
    text = re.sub(r'https?://\S+|www\.\S+|bit\.ly\S*', '', text)
    
    # 3. Remove standalone paths without protocols (e.g., "/content/kan-news/defense/629514/")
    text = re.sub(r'/\S+', '', text)
    
    # 4. Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # 5. Remove emojis
    text = emoji.replace_emoji(text, replace='')
    
    # 6. Remove diacritics
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(arabic_diacritics, '', text)
    
    # 7. Remove Tatweel (ـ)
    text = re.sub(r'ـ', '', text)
    
    # 8. Normalize Arabic text
    text = re.sub(r'[إأآا]', 'ا', text)  # Unify Alif variants
    text = re.sub(r'ة', 'ه', text)  # Replace Taa Marbuta with Haa
    text = re.sub(r'ى', 'ي', text)  # Replace Alef Maqsura with Ya
    
    # 9. Remove repeated characters (e.g., "ممتتتاز" → "ممتاز")
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Return cleaned text
    return text

In [18]:
# Apply the preprocessing function
df['Arabic MT'] = df['Arabic MT'].apply(preprocess_text)

In [19]:
df['Arabic MT'][9022]

'اولا علي قناه ABC: جلست المرشحه الرئاسيه الجمهوريه نيكي هيلي مع لينسي ديفيس من قناه ABC News، حيث ناقشت مجموعه واسعه من المواضيع بما في ذلك الرئيس السابق. ترامب والحرب بين اسرائيل وحماس والاجهاض وحياتها قبل ان تظهر امام الجمهور. اقرا المزيد: '

# Handling imbalanced dataset

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np

model = SentenceTransformer("intfloat/multilingual-e5-large")

X = df['Arabic MT']
y = df['Bias']

X_embeddings = model.encode(X.tolist(), convert_to_numpy=True)

X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, stratify=y, random_state=42)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/338 [00:00<?, ?it/s]

In [21]:
from imblearn.over_sampling import BorderlineSMOTE

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train, y_train)

print("Class distribution after Borderline-SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Class distribution after Borderline-SMOTE:
Bias
Unbiased                                    5454
Biased against Palestine                    5454
Unclear                                     5454
Biased against Israel                       5454
Biased against others                       5454
Biased against both Palestine and Israel    5454
Not Applicable                              5454
Name: count, dtype: int64


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

original_embeddings = X_train
synthetic_embeddings = X_train_resampled[len(X_train):]

cos_sim = cosine_similarity(original_embeddings, synthetic_embeddings)
average_similarity = np.mean(np.max(cos_sim, axis=0))
print(f"Average Cosine Similarity of Synthetic Samples: {average_similarity:.4f}")

if average_similarity < 0.7:
    print("Warning: Synthetic samples are significantly different from the original data.")
else:
    print("Synthetic samples appear similar to the original data.")

Average Cosine Similarity of Synthetic Samples: 0.9886
Synthetic samples appear similar to the original data.


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

# Splitting the data into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

# Initializing Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

# Fitting the Logistic Regression model
log_reg_model.fit(X_train_final, y_train_final)

# Making predictions
y_pred = log_reg_model.predict(X_val)

# Evaluating the model
print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Calculating F1 Score (Macro and Weighted)
f1_macro = f1_score(y_val, y_pred, average='macro')
f1_weighted = f1_score(y_val, y_pred, average='weighted')

print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Weighted): {f1_weighted:.4f}")

Classification Report:
                                          precision    recall  f1-score   support

                   Biased against Israel       0.82      0.91      0.86      1082
                Biased against Palestine       0.59      0.53      0.55      1108
Biased against both Palestine and Israel       0.97      1.00      0.98      1099
                   Biased against others       0.90      0.91      0.90      1078
                          Not Applicable       0.94      0.96      0.95      1079
                                Unbiased       0.53      0.48      0.50      1107
                                 Unclear       0.78      0.78      0.78      1083

                                accuracy                           0.79      7636
                               macro avg       0.79      0.80      0.79      7636
                            weighted avg       0.79      0.79      0.79      7636

Confusion Matrix:
[[ 990   28    4    2    4   39   15]
 [  70  582   11